Chapter05 완전 연결 신경망 분류(Classification)

15 원-핫 인코딩과 교차 엔트로피 오차

16 활성화 함수

17 분류 성능평가

18 1-Dense 층(1뉴런) AND?OR 분류

19 1-Dense 층(2뉴런) AND?OR 분류

20 2층 신경망: XOR 이진 분류

21 2D 정규분포 데이터 생성 및 분류

22 IRIS 데이터 분류

# 0. Prerequisite

In [ ]:
# https://research.google.com/colaboratory/local-runtimes.html

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Step 16. Activation Function


* 우리 뇌의 뉴런은 임계치를 넘는 신호가 들어오면 출력함
* <a href="https://uploads-ssl.webflow.com/614c82ed388d53640613982e/64a6b855e23a0a3dff98bfa0_linear%20and%20non%20linear%20functions.webp">nonlinear</a>: 복잡한 표현을 학습하기 위해 필요
* 그렇지 않으면, 결국 레이어 하나로 표현 가능해짐
* weighted sum -> activation 함수의 입력 값

$$ Weighted\ Sum = \sum_{i=1}^nx_iW_i + b$$

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*Uzraphxv6aozpVYbqYYAVw.jpeg" width=800 />


In [ ]:
X = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]], dtype = np.float32)
y_true = np.array([[0],[0],[0],[1]], dtype = np.float32)   # AND
X, y_true

(array([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]], dtype=float32),
 array([[0.],
        [0.],
        [0.],
        [1.]], dtype=float32))

In [ ]:
model = tf.keras.Sequential()
# activation=tf.keras.activations.sigmoid
# sigmoid, tanh, relu, softmax
model.add(tf.keras.layers.Dense(units=1, input_dim=2, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 1)                 3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*a04iKNbchayCAJ7-0QlesA.png" width=400/>


### sigmoid

* 미분 가능함
* <a href="https://upload.wikimedia.org/wikipedia/commons/d/dc/Exponenciala_priklad.png">지수함수</a>로 (0, 1) 사이의 값 (마치 확률처럼)
* 함수값이 (0, 1) 값이므로 <a href="https://uploads-ssl.webflow.com/614c82ed388d53640613982e/64a7fb7b268ea3f26d08b456_vanishing-and-exploding-gradients.webp">exploding</a> 안됨
* but, <a href="https://uploads-ssl.webflow.com/614c82ed388d53640613982e/64a7fb7b268ea3f26d08b456_vanishing-and-exploding-gradients.webp">vanishing gradient</a> 문제가 있음
* weights sum 값이 -100이나 100인 경우 gradient가 거의 0에 가까워서 saturated neurons 은 거의 변경되지 않음




## Gradient vanishing, exploding, saturation problem

<img src="https://global-uploads.webflow.com/5ef788f07804fb7d78a4127a/6245a9aca7defe61cea5ea7d_Engati-vanishing-point-problem.jpg" width=400><img src="https://miro.medium.com/max/1178/1*54WxyNRIoO6OX8Jwa-caeQ.webp" width=400>



* gradient vanishing problem, 미분값 최대 0.25,
* layer가 7층인 경우, $0.25^7 = 0.000061035 $


<img src="https://miro.medium.com/max/1400/1*ZafDv3VUm60Eh10OeJu1vw.webp" width=800 />

### tanh

* scaled sigmoid function
* 함수값이 (-1,1) 사이고, (0,0) 에서 대칭
* RNN, LSTM

### relu (hinton, 2010)

* vanishing gradient 문제를 해결하고자 고안 (z 값이 0 보다 크면 미분값이 1임)
* 함수값은 max(0, z)
* 가장 널리 사용됨 (특히 큰 규모의 모델에서)
* 계산이 빠름
* but, [0, 무한대) 범위, 양수값이 너무 커서 exploding 될 수 있음
* relu 는 마이너스에서 미분 불가
* weight sum 값이 마이너스이면 0 이 됨 => dying ReLU 문제 (미분값이 없으므로, weight 초기값에 변화가 없음)


### leaky relu(2013)

* relu 가 마이너스에서 미분되지 않는 단점을 보완
* max(0.1z, z)

### softmax

* 주로 분류(classification) 모델의 마지막 계층의 활성화 함수로 사용됨




<img src="https://www.researchgate.net/profile/Nabi-Nabiyev-2/publication/349662206/figure/fig3/AS:995882686246913@1614448343589/Working-principles-of-softmax-function.jpg" width=400/>

$$ softmax(z) = \frac{{e^{z_{j}}}}{\Sigma_{n=1}^{n}{e^{z_{n}}}}, (for \ j=1, 2, ...n)$$

* 큰 값을 강조하고 작은 값을 약화시키는 효과
* 0 이나 음수에도 적용 가능
* 결과값으로 확률이 필요한 경우 사용 (classification 등)

In [ ]:
#z=tf.random.normal(shape=(1,3))
z=[2.,1,0]
z=tf.reshape(z, shape=(1,3))
print('z', z)

softmax_output=tf.keras.activations.softmax(z)
print('softmax_output', softmax_output)

# sigmoid_output=tf.keras.activations.sigmoid(z)
# print('sigmoid_output', sigmoid_output)

z tf.Tensor([[2. 1. 0.]], shape=(1, 3), dtype=float32)
softmax_output tf.Tensor([[0.66524094 0.24472848 0.09003057]], shape=(1, 3), dtype=float32)


In [ ]:
import tensorflow as tf
import numpy as np

x = tf.constant([-10, -1.0, 0.0, 1.0, 10], dtype = tf.float32)

y1 = tf.keras.activations.linear(x)
y2 = tf.keras.activations.sigmoid(x)
y3 = tf.keras.activations.tanh(x)
y4 = tf.keras.activations.relu(x)
y5 = tf.keras.layers.LeakyReLU(alpha=0.1)(x)
y6 = tf.keras.activations.softmax(tf.reshape(x, shape=(1, -1)))

print("y1=", y1.numpy())
print("y2=", y2.numpy())
print("y3=", y3.numpy())
print("y4=", y4.numpy())
print("y5=", y5.numpy())
print("y6=", y6.numpy())
print("sum(y6)=", np.sum(y6.numpy())) # 1.0

y1= [-10.  -1.   0.   1.  10.]
y2= [4.5397868e-05 2.6894143e-01 5.0000000e-01 7.3105860e-01 9.9995458e-01]
y3= [-1.        -0.7615942  0.         0.7615942  1.       ]
y4= [ 0.  0.  0.  1. 10.]
y5= [-1.  -0.1  0.   1.  10. ]
y6= [[2.0607716e-09 1.6698603e-05 4.5391513e-05 1.2338691e-04 9.9981457e-01]]
sum(y6)= 1.0


# Step 15. One-hot encoding

숫자가 아닌 데이터([개, 고양이], 문자열 - "이것을 강력 추천합니다" 등) 를 모델의 입력으로 사용하려면 숫자로 변경해야 함

### why one-hot encoding?

개, 고양이, 토끼를 각각 1,2,3 으로 인코딩 하면,

* 토끼가 가장 중요한 값인가?

* 개와 토끼의 평균은 고양이인가?




### when should not use?

* 카테고리 값이 50개이면, 새로운 컬럼 50개가 추가됨

* 남 10, 여 01 로 one-hot 인코딩하면, 두 컬럼 사이에 강한 상관 관계가 발생함 (남 컬럼이 1이면, 무조건 여 컬럼은 0임)

| 원래 컬럼   |   남 컬럼      |  여 컬럼 |
|----------|:-------------:|------:|
| 남자 |  1 | 0 |
| 여자 |  0 | 1 |
| 남자 |  1 | 0 |


## p.113

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical

import tensorflow as tf
import numpy as np

y = range(2) # integer label
print("y=", list(y))

y = np.arange(2)
y1 = tf.keras.utils.to_categorical(y) # keras one-hot label
print("y1=", y1)

y2 = tf.keras.utils.to_categorical([1, 2, 3, 0, 2], num_classes=4)
print("y2=", y2)

y= [0, 1]
y1= [[1. 0.]
 [0. 1.]]
y2= [[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]



## Categorical Cross-Entropy Loss

https://en.wikipedia.org/wiki/Entropy_%28information_theory%29

> In information theory, the entropy of a random variable is the average level of "information", "surprise", or "uncertainty" inherent to the variable's possible outcomes.


$$ Entropy(x) = log \frac{1}{softmax(x)} = -log(softmax(x)) $$

<br>

$$ Entropy의\ 기대값 (x) = 확률(Y\_label)에\ 엔트로피를\ 곱한\ 값 \\
즉,\ E(엔트로피)\ =\ -확률(Y\_label)\ log(softmax(x)) $$

* 엔트로피는 불확실성이 높으면 값이 높음
* 엔트로피는 확률의 역수에 로그를 취한 값
* 확률의 역수를 취하는 이유는 확률이 높은 사건일수록 정보량(놀라움, 불확실성)이 낮다고 판단
* loss 로 사용 시 엔트로피 값 (불확실성) 을 낮추는 방향으로 학습


$$ CCE = -\frac{1}{n}\Sigma_{i=1}^nE(x)$$



In [ ]:
softmax_output=[[0.87, 0.13], [0.6, 0.4]]
entropy=tf.math.log(softmax_output)
print('entropy', entropy)

y_label=tf.keras.utils.to_categorical([0,1], num_classes=2)
print('y_label', y_label)

E_entropy = -1/2*(y_label * entropy)
print('E_entropy', E_entropy)

CCE = tf.reduce_sum(E_entropy)
print('CCE', CCE)

tf.keras.losses.CategoricalCrossentropy()(y_label, softmax_output)

entropy tf.Tensor(
[[-0.13926207 -2.040221  ]
 [-0.5108256  -0.9162907 ]], shape=(2, 2), dtype=float32)
y_label [[1. 0.]
 [0. 1.]]
E_entropy tf.Tensor(
[[0.06963103 0.        ]
 [0.         0.45814535]], shape=(2, 2), dtype=float32)
CCE tf.Tensor(0.52777636, shape=(), dtype=float32)


<tf.Tensor: shape=(), dtype=float32, numpy=0.52777636>

### CategoricalCrossentropy

In [ ]:
import tensorflow as tf
import numpy as np

CCE = tf.keras.losses.CategoricalCrossentropy()

# one-hot encoding
y_label= np.array([[1,   0,   0,   0],   #t[0]
             [0,   1,   0,   0],   #t[1]
             [0,   0,   1,   0],   #t[2]
             [0,   0,   0,   1]])  #t[3]

# softmax output
z =np.array([[0.4, 0.3, 0.2, 0.1], #y[0]
             [0.1, 0.3, 0.2, 0.4]])#y[1]

#1
print("CCE(t[i], y[0])")
print("CCE(t[0], y[0])=", CCE(y_label[0], z[0]).numpy() )
print("CCE(t[1], y[0])=", CCE(y_label[1], z[0]).numpy() )
print("CCE(t[2], y[0])=", CCE(y_label[2], z[0]).numpy() )
print("CCE(t[3], y[0])=", CCE(y_label[3], z[0]).numpy() )

#2
print("CCE(t[i], y[1])")
print("CCE(t[0], y[1])=", CCE(y_label[0], z[1]).numpy() )
print("CCE(t[1], y[1])=", CCE(y_label[1], z[1]).numpy() )
print("CCE(t[2], y[1])=", CCE(y_label[2], z[1]).numpy() )
print("CCE(t[3], y[1])=", CCE(y_label[3], z[1]).numpy() )

#3
print("CCE(np.vstack((t[1], t[1])), y)=",
       CCE(np.vstack((y_label[0], y_label[1])), z).numpy() )

(0.916290731874155 + 1.203972804325936) / 2

CCE(t[i], y[0])
CCE(t[0], y[0])= 0.916290731874155
CCE(t[1], y[0])= 1.203972804325936
CCE(t[2], y[0])= 1.6094379124341003
CCE(t[3], y[0])= 2.3025850929940455
CCE(t[i], y[1])
CCE(t[0], y[1])= 2.3025850929940455
CCE(t[1], y[1])= 1.203972804325936
CCE(t[2], y[1])= 1.6094379124341003
CCE(t[3], y[1])= 0.916290731874155
CCE(np.vstack((t[1], t[1])), y)= 1.0601317681000455


1.0601317681000455

### SparseCategoricalCrossentropy

In [ ]:
import tensorflow as tf
import numpy as np

SCE = tf.keras.losses.SparseCategoricalCrossentropy()

# label encoding
t = tf.convert_to_tensor([0, 1, 2, 3])

# softmax output
y =tf.convert_to_tensor([[0.4, 0.3, 0.2, 0.1], #y[0]
                         [0.1, 0.3, 0.2, 0.4]])#y[1]

#1
print("SCE(t[i], y[0])")
print("SCE(t[0], y[0])=", SCE(t[0], y[0]).numpy() )
print("SCE(t[1], y[0])=", SCE(t[1], y[0]).numpy() )
print("SCE(t[2], y[0])=", SCE(t[2], y[0]).numpy() )
print("SCE(t[3], y[0])=", SCE(t[3], y[0]).numpy() )

#2
print("SCE(t[i], y[1])")
print("SCE(t[0], y[1])=", SCE(t[0], y[1]).numpy() )
print("SCE(t[1], y[1])=", SCE(t[1], y[1]).numpy() )
print("SCE(t[2], y[1])=", SCE(t[2], y[1]).numpy() )
print("SCE(t[3], y[1])=", SCE(t[3], y[1]).numpy() )

#3
print("SCE(tf.stack((t[1], t[1])), y)=",
       SCE(tf.stack((t[0], t[1])), y).numpy() )

SCE(t[i], y[0])
SCE(t[0], y[0])= 0.91629076
SCE(t[1], y[0])= 1.2039728
SCE(t[2], y[0])= 1.609438
SCE(t[3], y[0])= 2.3025851
SCE(t[i], y[1])
SCE(t[0], y[1])= 2.3025851
SCE(t[1], y[1])= 1.2039728
SCE(t[2], y[1])= 1.609438
SCE(t[3], y[1])= 0.91629076
SCE(tf.stack((t[1], t[1])), y)= 1.0601318


### BinaryCrossentropy

In [ ]:
import tensorflow as tf
import numpy as np

BCE = tf.keras.losses.BinaryCrossentropy()

# one-hot encoding
t= np.array([[1,   0,   0,   0],   #t[0]
             [0,   1,   0,   0],   #t[1]
             [0,   0,   1,   0],   #t[2]
             [0,   0,   0,   1]])  #t[3]

# softmax output
y =np.array([[0.4, 0.3, 0.2, 0.1], #y[0]
             [0.1, 0.3, 0.2, 0.4]])#y[1]

#1
print("BCE(t[i], y[0])")
print("BCE(t[0], y[0])=", BCE(t[0], y[0]).numpy() )
print("BCE(t[1], y[0])=", BCE(t[1], y[0]).numpy() )
print("BCE(t[2], y[0])=", BCE(t[2], y[0]).numpy() )
print("BCE(t[3], y[0])=", BCE(t[3], y[0]).numpy() )

#2
print("BCE(t[i], y[1])")
print("BCE(t[0], y[1])=", BCE(t[0], y[1]).numpy() )
print("BCE(t[1], y[1])=", BCE(t[1], y[1]).numpy() )
print("BCE(t[2], y[1])=", BCE(t[2], y[1]).numpy() )
print("BCE(t[3], y[1])=", BCE(t[3], y[1]).numpy() )

#3
print("BCE(np.vstack((t[0], t[0])), y)=",
       BCE(np.vstack((t[0], t[0])), y).numpy() )

BCE(t[i], y[0])
BCE(t[0], y[0])= 0.4003672784541813
BCE(t[1], y[0])= 0.5108254397382338
BCE(t[2], y[0])= 0.6455745187904711
BCE(t[3], y[0])= 0.8483069443724252
BCE(t[i], y[1])
BCE(t[0], y[1])= 0.8483069443724252
BCE(t[1], y[1])= 0.5108254397382338
BCE(t[2], y[1])= 0.6455745187904711
BCE(t[3], y[1])= 0.40036727845418124
BCE(np.vstack((t[0], t[0])), y)= 0.6243371114133032


### 결론적으로 ...

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=1, input_dim=2, activation='softmax'))
model.summary()
# cross-entropy loss for binary (0 or 1) classification
model.compile(optimizer='adam', loss='binary_crossentropy')

# labels to be provided in a one_hot representation
# y_label = [[0, 1, 0], [1, 0, 0]]
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=3, input_dim=2, activation='softmax'))
model.summary()
# two or more label classes, one_hot representation
model.compile(optimizer='adam', loss='categorical_crossentropy')

# labels as integers
# y_label = [1, 2]
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=3, input_dim=2, activation='softmax'))
model.summary()
# two or more label classes, labels as integers
model.compile(optimizer='adam', loss='binary_crossentropy')

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 1)                 3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 3)                 9         
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)   

# Step 17. tf.keras.metrics for classification

<img src="https://2.bp.blogspot.com/-EvSXDotTOwc/XMfeOGZ-CVI/AAAAAAAAEiE/oePFfvhfOQM11dgRn9FkPxlegCXbgOF4QCLcBGAs/s1600/confusionMatrxiUpdated.jpg" width=400/><img src="https://1.bp.blogspot.com/-b_lbkGbwKCo/W-lK6MZ3-xI/AAAAAAAAAYA/uuAkY-KkyEAdSJPxJKRaam0qQoMo9wZ4ACLcBGAs/s1600/2.PNG" width=400/>



### 결론적으로 ...

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=1, input_dim=2, activation='softmax'))
model.summary()

### metrics=['accuracy']

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'f1_score'])

In [ ]:
print("y_true=",y_true)
print("y_pred=",y_pred)
C = tf.math.confusion_matrix(y_true, y_pred)
print("confusion_matrix(C)=", C)

y_true= [1 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 1]
y_pred= [0 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 1]
confusion_matrix(C)= tf.Tensor(
[[9 3]
 [3 3]], shape=(2, 2), dtype=int32)


In [ ]:
#5
#5-1
m = tf.keras.metrics.TruePositives()
m.update_state(y_true, y_pred)
tp = m.result()  # m.true_positives
print("tp =", tp.numpy())

#5-2
m = tf.keras.metrics.TrueNegatives()
m.update_state(y_true, y_pred)
tn = m.result() # m.accumulator[0]
print("tn=", tn.numpy())

#5-3
m = tf.keras.metrics.FalsePositives()
m.update_state(y_true, y_pred)
fp = m.result() # m.accumulator[0] sms
print("fp=", fp.numpy())

#5-4
m = tf.keras.metrics.FalseNegatives()
m.update_state(y_true, y_pred)
fn = m.result()# m.accumulator[0]
print("fn=", fn.numpy())

tp = 3.0
tn= 9.0
fp= 3.0
fn= 3.0


In [ ]:
accuracy4  = (tp + tn)/(tp+tn+fp+fn)
precision = tp/(tp+fp)
recall    = tp/(tp+fn)
f1 = 2*tp/(2*tp + fp + fn) # harmonic mean of precision and recall
print("accuracy4 =", accuracy4.numpy())
print("precision =",precision.numpy())
print("recall =",   recall.numpy())
print("f1 score =", f1.numpy())

accuracy4 = 0.6666667
precision = 0.5
recall = 0.5
f1 score = 0.5


In [ ]:
#4:
m = tf.keras.metrics.Accuracy()
m.update_state(y_true, y_pred)
print("m.total={}, m.count={}".format(m.total.numpy(), m.count.numpy()))
accuracy3 = m.result()  # m.total/m.count
print("accuracy3=", accuracy3.numpy())

#6
#6-1
m = tf.keras.metrics.Precision()
m.update_state(y_true, y_pred)
print("m.true_positives=", m.true_positives.numpy())
print("m.false_positives", m.false_positives.numpy())
print("precision=", m.result().numpy())

#6-2
m = tf.keras.metrics.Recall()
m.update_state(y_true, y_pred)
print("m.true_positives=", m.true_positives.numpy())
print("m.false_negatives", m.false_negatives.numpy())
print("recall=", m.result().numpy())

m.total=12.0, m.count=18.0
accuracy3= 0.6666667
m.true_positives= [3.]
m.false_positives [3.]
precision= 0.5
m.true_positives= [3.]
m.false_negatives [3.]
recall= 0.5


### p.129

In [ ]:
import tensorflow as tf
import numpy as np
#1
##y_true = np.array([0, 1, 2, 0, 1, 2])
##y_true = tf.keras.utils.to_categorical(y_true) # one-hot
y_true = np.array([[1, 0, 0], #0
                   [0, 1, 0], #1
                   [0, 0, 1], #2
                   [1, 0, 0], #0
                   [0, 1, 0], #1
                   [0, 0, 1]]);#2

y_pred = np.array([[0.3, 0.6, 0.1],  #1,
                   [0.6, 0.3, 0.1],  #0
                   [0.1, 0.3, 0.6],  #2
                   [0.3, 0.6, 0.1],  #1
                   [0.1, 0.6, 0.3],  #1
                   [0.3, 0.1, 0.6]]);#2
num_class = y_true.shape[1] # 3

#2: C and TOP_k
#2-1: threshold, and C in # 3-1, #4-1, and #6 in [step1701]
y_true1 = np.argmax(y_true, axis=1).flatten()
y_pred1 = np.argmax(np.cast['int'](y_pred>0.5), axis=1).flatten()
C = tf.math.confusion_matrix(y_true1, y_pred1)
print("y_true1=",y_true1) # y_true1= [0 1 2 0 1 2]
print("y_pred1=",y_pred1) # y_pred1= [1 0 2 1 1 2]
print("confusion_matrix(C)=", C)

#2-2: to find top-k index, in #3-2, #4-2
k=2
indx = tf.argsort(y_pred, axis=1, direction='DESCENDING')
TOP_k = indx[:,:k]
print("TOP_k = ", TOP_k)

#3
print("In each class, precision!")
#3-1: binary(1 above threshold=0.5, 0 below threshold= 0.5)
for i in range(num_class):
    m = tf.keras.metrics.Precision(class_id = i)
    m.update_state(y_true, y_pred)
    tp = m.true_positives.numpy()
    fp = m.false_positives.numpy()
    p = m.result().numpy()
    print(" p_{} ={}, tp={}, fp= {}".format(i,p, tp, fp))

#3-2: the top-k classes with the highest predicted values
print("In each class, precision with top_k=", k)
for i in range(num_class):
    m = tf.keras.metrics.Precision(top_k=k, class_id = i)
    m.update_state(y_true, y_pred)
    tp = m.true_positives.numpy()
    fp = m.false_positives.numpy()
    p = m.result().numpy()
    print(" p_{} ={}, tp={}, fp= {}".format(i,p, tp, fp))
#4
print("In each class, recall!")
#4-1: binary(1 above threshold=0.5, 0 below threshold= 0.5)
for i in range(num_class):
    m = tf.keras.metrics.Recall(class_id = i)
    m.update_state(y_true, y_pred)
    tp = m.true_positives.numpy()
    fn = m.false_negatives.numpy()
    r = m.result().numpy()
    print(" recall_{} ={}, tp={}, fn= {}".format(i,r, tp, fn))

#4-2: the top-k classes with the highest predicted values
print("In each class, recall with top_k=", k)
for i in range(num_class):
    m = tf.keras.metrics.Recall(top_k=k, class_id = i)
    m.update_state(y_true, y_pred)
    r = m.result().numpy()
    print(" recall_{} ={}, tp={}, fn= {}".format(i,r, tp, fn))

y_true1= [0 1 2 0 1 2]
y_pred1= [1 0 2 1 1 2]
confusion_matrix(C)= tf.Tensor(
[[0 2 0]
 [1 1 0]
 [0 0 2]], shape=(3, 3), dtype=int32)
TOP_k =  tf.Tensor(
[[1 0]
 [0 1]
 [2 1]
 [1 0]
 [1 2]
 [2 0]], shape=(6, 2), dtype=int32)
In each class, precision!
 p_0 =0.0, tp=[0.], fp= [1.]
 p_1 =0.3333333432674408, tp=[1.], fp= [2.]
 p_2 =1.0, tp=[2.], fp= [0.]
In each class, precision with top_k= 2
 p_0 =0.5, tp=[2.], fp= [2.]
 p_1 =0.4000000059604645, tp=[2.], fp= [3.]
 p_2 =0.6666666865348816, tp=[2.], fp= [1.]
In each class, recall!
 recall_0 =0.0, tp=[0.], fn= [2.]
 recall_1 =0.5, tp=[1.], fn= [1.]
 recall_2 =1.0, tp=[2.], fn= [0.]
In each class, recall with top_k= 2
 recall_0 =1.0, tp=[2.], fn= [0.]
 recall_1 =1.0, tp=[2.], fn= [0.]
 recall_2 =1.0, tp=[2.], fn= [0.]


### p.132

In [ ]:
import tensorflow as tf
import numpy as np

#1
y_true = np.array([0, 1, 2, 0, 1, 2])
y_pred = np.array([1, 0, 2, 1, 1, 2])

#2
m = tf.keras.metrics.Accuracy()
m.update_state(y_true, y_pred)  #m.count = 3, m.total=6
print("accuracy from f.keras.metrics.Accuracy()=", m.result().numpy() )

#3
C = tf.math.confusion_matrix(y_true, y_pred)
print("confusion_matrix=", C)

correct = tf.linalg.diag_part(C)
col_sum = tf.reduce_sum(C, axis=0)
row_sum = tf.reduce_sum(C, axis=1)
total   = tf.reduce_sum(C)  #  len(y_true), len(y_pred)

accuracy    = tf.reduce_sum(correct)/total
precision_i = correct/col_sum
recall_i    = correct/row_sum
f1_i = 2*(precision_i*recall_i)/(precision_i+recall_i) # harmonic mean of precision and recall
f1_i = tf.where(tf.math.is_nan(f1_i), tf.zeros_like(f1_i), f1_i) # nan to 0.0
print("accuracy=",    accuracy.numpy())
print("precision_i=", precision_i.numpy())
print("recall_i=",    recall_i.numpy())
print("f1_i=",    f1_i.numpy())

#4:  micro, macro, weighted avg in precision, recall
tp = tf.reduce_sum(correct)    # notice : correct pairs such as (0,0), (1,1), (2,2)
fp = tf.reduce_sum(col_sum - correct) # in this case, fp == fn
fn = tf.reduce_sum(row_sum - correct)
precision = tp/(tp + fp)
recall    = tp/(tp + fn)

count = tf.math.bincount(y_true) # support  in sklearn.metrics
print("count =", count)
print("precision(micro avg)=", precision.numpy())
print("precision(macro avg)=", tf.reduce_sum(precision_i)/precision_i.shape[0])
w=  tf.cast(count, dtype = tf.float64)/y_true.shape[0]  # tf.cast(total, dtype = tf.float64)
weightedAvgP = tf.reduce_sum(precision_i*w)
print("precision(weighted avg)=", weightedAvgP)

print("recall(micro avg)=", recall.numpy())
print("recall(macro avg)=", tf.reduce_sum(recall_i)/recall_i.shape[0])
weightedAvgR = tf.reduce_sum(recall_i*w)
print("recall(weighted avg)=", weightedAvgR)

#5: pip install sklearn
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score,precision_score, recall_score
print("--- sklearn.metrics ---")
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))

print("accuracy=", accuracy_score(y_true, y_pred)) # normalize=True
print("precision_i=", precision_score(y_true, y_pred, average=None))
print("precision(micro avg)=", precision_score(y_true, y_pred, average='micro'))
print("precision(macro avg)=", precision_score(y_true, y_pred, average='macro'))

print("recall_i=", recall_score(y_true, y_pred, average=None))
print("recall(micro avg)=", recall_score(y_true, y_pred, average='micro'))
print("recall(macro avg)=", recall_score(y_true, y_pred, average='macro'))

accuracy from f.keras.metrics.Accuracy()= 0.5
confusion_matrix= tf.Tensor(
[[0 2 0]
 [1 1 0]
 [0 0 2]], shape=(3, 3), dtype=int32)
accuracy= 0.5
precision_i= [0.         0.33333333 1.        ]
recall_i= [0.  0.5 1. ]
f1_i= [0.  0.4 1. ]
count = tf.Tensor([2 2 2], shape=(3,), dtype=int32)
precision(micro avg)= 0.5
precision(macro avg)= tf.Tensor(0.4444444444444444, shape=(), dtype=float64)
precision(weighted avg)= tf.Tensor(0.4444444444444444, shape=(), dtype=float64)
recall(micro avg)= 0.5
recall(macro avg)= tf.Tensor(0.5, shape=(), dtype=float64)
recall(weighted avg)= tf.Tensor(0.5, shape=(), dtype=float64)
--- sklearn.metrics ---
[[0 2 0]
 [1 1 0]
 [0 0 2]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.33      0.50      0.40         2
           2       1.00      1.00      1.00         2

    accuracy                           0.50         6
   macro avg       0.44      0.50      0.47         6
weight

### p.135

In [ ]:
import tensorflow as tf
import numpy as np
#1
##y_true = np.array([0, 1, 2, 0, 1, 2])
##y_true = tf.keras.utils.to_categorical(y_true) # one-hot
y_true = np.array([[1, 0, 0], #0
                   [0, 1, 0], #1
                   [0, 0, 1], #2
                   [1, 0, 0], #0
                   [0, 1, 0], #1
                   [0, 0, 1]], dtype=np.int32);#2

y_pred = np.array([[0.3, 0.6, 0.1], #1
                   [0.6, 0.3, 0.1], #0
                   [0.1, 0.3, 0.6], #2
                   [0.3, 0.6, 0.1], #1
                   [0.1, 0.6, 0.3], #1
                   [0.3, 0.1, 0.6]], dtype=np.float32);#2

print(y_true.dtype)
print(y_pred.dtype)

#2: using one-hot encoding in y_true
print("CategoricalAccuracy!")

#2-1
accuracy2_1= tf.keras.metrics.categorical_accuracy(y_true, y_pred)
print("accuracy2_1=", accuracy2_1.numpy())
#2-2
m = tf.keras.metrics.CategoricalAccuracy()
m.update_state(y_true, y_pred)
# m.total = tf.reduce_sum(accuracy2_1)
# m.count = accuracy2_1.shape[0]
accuracy2_2 = m.result() # m.total/m.count
print("m.total={}, m.count={}".format(m.total.numpy(),m.count.numpy()))
print("accuracy2_2=", accuracy2_2.numpy())

#2-3
top_k = 2
accuracy2_3 = tf.keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k=top_k)
print("top_k={}, accuracy2_3={}".format(top_k, accuracy2_3))

#2-4
m = tf.keras.metrics.TopKCategoricalAccuracy(k=top_k) # default k = 5
m.update_state(y_true, y_pred)
# m.total = tf.reduce_sum(accuracy2_3)
# m.count = accuracy2_3.shape[0]
accuracy2_4 = m.result()
print("m.total={}, m.count={}".format(m.total.numpy(),m.count.numpy()))
print("top_k={}, accuracy2_4={}".format(top_k, accuracy2_4.numpy()))

#3: using integer label in y_true
print("SparseCategoricalAccuracy!")
y_true = tf.argmax(y_true, axis = 1) # np.argmax(y_true, axis = 1)
y_true = tf.reshape(y_true, (-1,1))  # np.reshape(y_true, (-1, 1))
print("y_true=", y_true)

#3-1
accuracy3_1= tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)
print("accuracy3_1=", accuracy3_1.numpy())
#3-2
m = tf.keras.metrics.SparseCategoricalAccuracy()
m.update_state(y_true, y_pred)
# m.total = tf.reduce_sum(accuracy3_1)
# m.count = accuracy3_1.shape[0]
accuracy3_2 = m.result() # m.total/m.count
print("m.total={}, m.count={}".format(m.total.numpy(),m.count.numpy()))
print("accuracy3_2=", accuracy3_2.numpy())

#3-3
top_k = 2
accuracy3_3 = tf.keras.metrics.sparse_top_k_categorical_accuracy(y_true, y_pred, k=top_k)
print("top_k={}, accuracy3_3={}".format(top_k, accuracy3_3))

#3-4
m = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=top_k) # default k = 5
m.update_state(y_true, y_pred)
# m.total = tf.reduce_sum(accuracy3_3)
# m.count = accuracy3_3.shape[0]
accuracy3_4 = m.result()
print("m.total={}, m.count={}".format(m.total.numpy(),m.count.numpy()))
print("top_k={}, accuracy3_4={}".format(top_k, accuracy3_4.numpy()))

int32
float32
CategoricalAccuracy!
accuracy2_1= [0. 0. 1. 0. 1. 1.]
m.total=3.0, m.count=6.0
accuracy2_2= 0.5
top_k=2, accuracy2_3=[1. 1. 1. 1. 1. 1.]
m.total=6.0, m.count=6.0
top_k=2, accuracy2_4=1.0
SparseCategoricalAccuracy!
y_true= tf.Tensor(
[[0]
 [1]
 [2]
 [0]
 [1]
 [2]], shape=(6, 1), dtype=int64)
accuracy3_1= [0. 0. 1. 0. 1. 1.]
m.total=3.0, m.count=6.0
accuracy3_2= 0.5
top_k=2, accuracy3_3=[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
m.total=6.0, m.count=6.0
top_k=2, accuracy3_4=1.0
